# Preamble

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
%cd drive/MyDrive/python/BayesianFiltering

[Errno 2] No such file or directory: 'drive/MyDrive/python/BayesianFiltering'
/content/drive/MyDrive/python/intent_detection


In [15]:
from ssm import StateSpaceModel
from ssm import LinearModelParameters
from SLDS import SLDS
from simulation import Simulation
from ssm import LGSSM
import numpy as np
import copy
import matplotlib.pyplot as plt

# Data Generation

In [ ]:
M = 2
dx = 2
dy = 2
T = 100
model_parameter_array = np.empty([M], dtype=LinearModelParameters)
init_state = np.zeros([dx])
# var_max = 10
# correl_mask_dx = np.ones([dx, dx]) - np.eye(dx)
# for m in range(M):
#     A = np.random.rand() * np.eye(dx)
#     H = 1 * np.eye(dy, dx)
#     #Q_nonsym = var_max * np.multiply(np.random.random([dx, dx]), np.eye(dx)) + \
#     #          np.multiply(np.random.random([dx, dx]), correl_mask_dx)
#     #Q = 1 * (Q_nonsym + Q_nonsym.T)
#     Q = 0.1 * np.random.rand() * np.eye(dx)
#     R = 1000 * np.random.rand() * np.eye(dy)
#     model_parameter_array[m] = LinearModelParameters(A, H, Q, R)

# Slow forgetting
A0 = -1 * np.eye(dx)
H0 = 1 * np.eye(dy, dx)
Q0 = 1 * np.eye(dx) #np.array([[10, 0] , [ 0, 0.1]])
R0 = 1 * np.eye(dy)

# Fast forgetting
theta = np.pi*5/360
A1 = 1 * np.eye(dx) # 1 * np.array([[np.cos(theta), -np.sin(theta)] , [ np.sin(theta), np.cos(theta)]])
H1 = 1 * np.eye(dy, dx)
Q1 = 0.01 * np.eye(dx) # np.array([[0.1, 0] , [ 0, 10]])
R1 = 5  * np.eye(dy)

model_parameter_array[0] = LinearModelParameters(A0, H0, Q0, R0)
model_parameter_array[1] = LinearModelParameters(A1, H1, Q1, R1)

SLDS1 = SLDS(dx, dy, model_parameter_array)

# alpha = np.random.choice(range(1, 50), M)
# mat = np.random.dirichlet(alpha, M) # Random tranisiton matrix with Dirichlet(alpha) rows
mat = np.array([[0.8, 0.2] , [ 0.2, 0.8]])

SLDS1.set_transition_matrix(mat)

sim3 = Simulation(SLDS1, T, init_state=[0, init_state])

# Missing data
obs_interval = 10
for t in range(T):
  if t%obs_interval:
    sim3.observs[t] = None

# Filtering

### Conditional

In [ ]:
filt_slds_model = copy.deepcopy(SLDS1)
dx_slds = filt_slds_model.dx
init_slds = [np.zeros(dx_slds), np.eye(dx_slds)]
out_exact_cond = filt_slds_model.conditional_kalman_filter(sim3.states[0], sim3.observs, init_slds)
# static_cond = np.zeros([M, T])
# for m in range(M):
#   hist = m*np.ones(T, dtype=int)
#   out1 = filt_slds_model.conditional_kalman_filter(hist, sim3.observs, init_slds)
#   static_cond[m] = np.squeeze(out1[0])

### IMM

In [ ]:
filt_slds_model = copy.deepcopy(SLDS1)
dx_slds = filt_slds_model.dx
init_slds = [np.zeros(dx_slds), np.eye(dx_slds)]
mean_out_IMM, cov_out_IMM, weights_out_IMM = filt_slds_model.IMM(sim3.observs, init_slds)

ValueError: ignored

### GPB

In [ ]:
filt_slds_model2 = copy.deepcopy(SLDS1)
dx_slds = filt_slds_model2.dx
init_slds = [np.zeros(dx_slds), np.eye(dx_slds)]

R = 9
meansGPB = np.zeros([R-1, T])
for r in range(2,R):
  mean_out_GPB, cov_out_GPB, weights_out_GPB = filt_slds_model2.GPB(r, sim3.observs, init_slds)
  meansGPB[r-2] = np.squeeze(mean_out_GPB)

### Error Calculations

In [ ]:
errGPB_trueStates = np.zeros(T)
errIMM_trueStates = np.zeros(T)
errGPB_ExCond = np.zeros(T-1)
errIMM_ExCond = np.zeros(T-1)
for t in range(T):
    errGPB_trueStates[t] = np.linalg.norm(mean_out_GPB[t, :] - sim3.states[1][t, :])
    errGPB_ExCond[t-1] = np.linalg.norm(mean_out_GPB[t, :] - out_exact_cond[0][t-1, :])
    errIMM_trueStates[t] = np.linalg.norm(mean_out_IMM[t, :] - sim3.states[1][t, :])
    errIMM_ExCond[t-1] = np.linalg.norm(mean_out_IMM[t, :] - out_exact_cond[0][t-1, :])


# Plots 1D

In [ ]:
fig1, axes1 = plt.subplots(1, 1, sharex=True, figsize=(10, 4))
p1 = axes1.plot(sim3.states[1], alpha=0.6, label="States")
p2 = axes1.plot(mean_out_GPB, alpha=0.6, label="GPB")
p3 = axes1.plot(mean_out_IMM, alpha=0.6, label="IMM")
axes1.set_ylabel("X")
axes1.set_xlabel("time")
axes1.set_title("True States VS Est")
axes1.legend(['true', 'GPB', 'IMM'])

fig2, axes2 = plt.subplots(1, 1, sharex=True, figsize=(10, 4))
p1 = axes2.plot(out_exact_cond[0], alpha=0.6, label="States")
p2 = axes2.plot(mean_out_GPB, alpha=0.6, label="GPB")
p3 = axes2.plot(mean_out_IMM, alpha=0.6, label="IMM")
# axes2.plot(static_cond[0])
# axes2.plot(static_cond[1])
axes2.set_ylabel("X")
axes2.set_xlabel("time")
axes2.set_title("Ex Cond VS Est")
axes2.legend(['Ex Cond', 'GPB', 'IMM','Static 0', 'Static 1'])

fig3, axes3 = plt.subplots(2, 1, sharex=True, figsize=(10, 4))
p1 = axes3[0].plot(errGPB_trueStates, label="GPB")
p1 = axes3[0].plot(errIMM_trueStates, label="IMM")
axes3[0].set_ylabel("Error")
#axes3[0].set_xlabel("time")
axes3[0].set_title("Error comp to True States")
axes3[0].legend(["GPB", "IMM"])
plot1 = axes3[1].plot(errGPB_ExCond, label="GPB")
plot2 = axes3[1].plot(errIMM_ExCond, label="IMM")
axes3[1].set_ylabel("Error")
axes3[1].set_title("Error comp to Ex Cond")
axes3[1].legend(["GPB", "IMM"])

fig4, axes4 = plt.subplots(2, 1, sharex=True, figsize=(10, 4))
plot1 = axes4[0].plot(weights_out_GPB)
axes4[0].set_ylabel("Weight")
#axes4[0].set_xlabel("time")
axes4[0].set_title("Weights GPB")
plot = axes4[1].plot(weights_out_IMM)
axes4[1].set_ylabel("Weight")
axes4[1].set_xlabel("time")
axes4[1].set_title("Weights IMM")

fig5, axes5 = plt.subplots(2, 1, sharex=True, figsize=(10, 4))
axes5[0].plot(sim3.states[1])
axes5[0].plot(sim3.observs)
axes5[0].set_ylabel("x,y")
axes5[0].set_title("Simul")
axes5[0].legend(["states"])
axes5[1].plot(sim3.states[0])
axes5[1].set_ylabel("model")
axes5[1].set_xlabel("time")
axes5[1].set_title("True Model History")

fig6, axes6 = plt.subplots(1, 1, sharex=True, figsize=(10, 4))
axes6.plot(sim3.states[1], alpha=0.6, label="States")
for r in range(2,R):
  axes6.plot(meansGPB[r-2], alpha=0.6, label="GPB")
axes6.legend(['true']+["GPB{}".format(r) for r in range(2,R)])


plt.show()

# Plots nd


In [ ]:
# Plots
# Estimates and States
fig1, axes1 = plt.subplots(1, 1, sharex=True, figsize=(10, 4))
scatter1 = axes1.scatter(sim3.states[1][:, 0], sim3.states[1][:, 1], alpha=0.6, label="States")
scatter2 = axes1.scatter(mean_out_GPB[:, 0], mean_out_GPB[:, 1], alpha=0.6, marker = "v", label="GPB")
scatter3 = axes1.scatter(mean_out_IMM[:, 0], mean_out_IMM[:, 1], alpha=0.6, label="IMM")
axes1.set_ylabel("X2")
axes1.set_xlabel("X1")
axes1.set_title("True States VS Est")
axes1.legend(handles=[scatter1, scatter2, scatter3])

fig2, axes2 = plt.subplots(1, 1, sharex=True, figsize=(10, 4))
scatter1 = axes2.scatter(out_exact_cond[0][:, 0], out_exact_cond[0][:, 1], alpha=0.6, marker="v", label="exact cond")
scatter2 = axes2.scatter(mean_out_GPB[:, 0], mean_out_GPB[:, 1], alpha=0.6, marker="v", label="GPB")
scatter3 = axes2.scatter(mean_out_IMM[:, 0], mean_out_IMM[:, 1], alpha=0.6, label="IMM")
axes2.set_ylabel("X2")
axes2.set_xlabel("X1")
axes2.set_title("Exact Conditional VS Est")
axes2.legend(handles=[scatter1, scatter2, scatter3])

# Errors 
fig3, axes3 = plt.subplots(2, 1, sharex=True, figsize=(10, 4))
plot11 = axes3[0].plot(errGPB_trueStates, label="GPB")
plot21 = axes3[0].plot(errIMM_trueStates, label="IMM")
axes3[0].set_ylabel("Error")
axes3[0].set_title("Error comp to True States")

plot21 = axes3[1].plot(errGPB_ExCond, label="GPB")
plot22 = axes3[1].plot(errIMM_ExCond, label="IMM")
axes3[1].set_ylabel("Error")
axes3[1].set_xlabel("time")
axes3[1].set_title("Error comp to Ex Cond")

# Weights and Model History
fig4, axes4 = plt.subplots(2, 1, sharex=True, figsize=(10, 4))
plot1 = axes4[0].plot(weights_out_GPB)
axes4[0].set_ylabel("Weight")
#axes4[0].set_xlabel("time")
axes4[0].set_title("Weights GPB")

plot = axes4[1].plot(weights_out_IMM)
axes4[1].set_ylabel("Weight")
axes4[1].set_xlabel("time")
axes4[1].set_title("Weights IMM")

fig5, axes5 = plt.subplots(2, 1, sharex=True, figsize=(10, 4))
axes5[0].plot(sim3.states[0])
axes5[0].set_ylabel("model")
axes5[0].set_xlabel("time")
axes5[0].set_title("True Model History")
plt.show()

In [16]:
np.random.seed(0)
var_Q = 1
var_Delta = 0.99
mu = 0 
Nrep = 100
Nsim = 10000
I1 = np.zeros(Nrep)
I2 = np.zeros(Nrep)
for rep in range(Nrep):
  x = np.random.normal(mu, np.sqrt(var_Q), Nsim)
  z = np.random.normal(mu, np.sqrt(var_Q-var_Delta) , Nsim)
  I1[rep] = np.sum(np.exp(-x**4 / (2*var_Q))/ np.sqrt(2*np.pi*var_Q)) / Nsim
  I2[rep] = np.sum(np.exp(-z**4 / (2*(4*z**2*var_Delta + var_Q))) /
              (np.sqrt(2*np.pi*(4*z**2*var_Delta + var_Q)))) / Nsim
  
print(np.abs(np.mean(I1)-0.278353), np.var(I1))
print(np.abs(np.mean(I2)-0.278353), np.var(I2))

0.00012041633440862398 1.5645845469611927e-06
0.11325377835545486 7.521959126119058e-09
